__Ny modell. Nu delar vi upp datat på omgångarna, inte slumpmässigt. Totalt 222 omgångar. Bygger modell på 175 omgångar, validerar på de övriga. Imputerar för missing innan vi kör modellen, tar bort detta steg från data pipelinen. Vi läser även in VNUM som är rankingen efter spelade hästar för att jämföra med den framtagna modellen__

__Tar fram optimal modell med cross validering och en randomforrest. Först en modell bara på v75% och VNUM för att plocka 21 hästar. Sedan en andra modell som använder Leffes rankingar för att idenfiera de hästar som inte fångas u__

In [2]:
# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [3]:
from sklearn.model_selection import GridSearchCV 

In [4]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import cross_val_score

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
# Where to save the figures
PROJECT_ROOT_DIR = os.getcwd()
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR)

In [6]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [7]:
df0 = pd.read_excel('DataV75TillUffeS20190525.xlsx')

In [8]:
# Skapar en unik nyckel på lopp: Gör om Datum och lopp till en sträng

df0['cdate'] = df0.Datum.astype('object')
df0['cLopp'] = df0.Lopp.astype('object')

In [9]:
df0['Key'] = df0['cdate'].astype(str) + df0['cLopp'].astype(str)

__Skapar en målvariabel - vinnare__

In [10]:
df0['Y'] = np.where(df0['Plac'].isin([1]), 1,0)

In [11]:
df0.Datum.max()

Timestamp('2019-05-25 00:00:00')

__Plockar bort de variabler som inte ska med__

In [12]:
df1 = df0.copy(deep = True)
# Alla analysvariabler
df1 = df1. \
drop(['Utdelning','Plac','cdate','cLopp','V_ODDS','S_R', \
      'TK_R','Arstid','Distans','Startsatt','SP_R','Ex_R','R_R','P_R'], axis = 1). \
copy().set_index(['Key'])

KeyError: "['Utdelning' 'S_R'] not found in axis"

In [ ]:
# Iterera över varje kolumn och använd logiken nedan
# Checkar dessa variabler [G_R , A_R, ToR , P_R ]
# Skapar en lista som håller de unika datumen för dessa när missing förekommer
# Tar sedan bort dessa lopp och bygger modell på de kvarvarande

In [ ]:
df1['mflag1'] = np.where(df1.G_R.isna(),True,False)
df1['mflag2'] = np.where(df1.A_R.isna(),True,False)
df1['mflag3'] = np.where(df1.ToR.isna(),True,False)
#df1['mflag4'] = np.where(df1.P_R.isna(),True,False)


In [ ]:
#df_delete = df1[(df1.mflag1 | df1.mflag2 | df1.mflag3 | df1.mflag4)]

df_delete = df1[(df1.mflag1 | df1.mflag2 | df1.mflag3)]

In [ ]:
len(df_delete)

In [ ]:
date_filter = df_delete.Datum.drop_duplicates().tolist()

In [ ]:
df1 = df1[~df1.Datum.isin(date_filter)]

In [ ]:
df1.info()

In [ ]:
#df1 = df1.drop(['mflag1', 'mflag2','mflag3','mflag4'], axis = 1)
df1 = df1.drop(['mflag1', 'mflag2','mflag3'], axis = 1)

In [ ]:
# Gör om grupp till objekt
df1['GRUPP'] = df1.GRUPP.astype('object')


__Delar upp i numeriska samt charachter attribut. Det är dessa som går in i modellen__

In [ ]:
num_attribs = [] 
cat_attribs = [] 

for var, typ in zip(df1.columns[:-1], df1.dtypes[:-1]): 
    if typ == 'object': 
        cat_attribs.append(var) 
    elif (typ != 'datetime64[ns]')  & (var != 'Hast') & (var != 'Lopp'): 
        num_attribs.append(var)

In [ ]:
cat_attribs

In [ ]:
num_attribs

In [ ]:
len(num_attribs)

In [ ]:
num_attribs.append('V75PROC')

In [ ]:
num_attribs

__Nu bygger vi upp en pipeline__

In [ ]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
# Denna klass måste vi göra för att särskilja numeriska variabler mot character variabler
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [ ]:
# Egen klass för att sätta dummyvariabler

class SetDummyVar(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        tempdf = pd.get_dummies(X[self.attribute_names], columns = self.attribute_names)
        return tempdf.values

In [ ]:
# Pipeline för numeriska variabler
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', Imputer(strategy="median"))
    ])

cat_pipeline = Pipeline([
        ('dummy_cat', SetDummyVar(cat_attribs)),
    ])

In [ ]:
full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
    ])

__Nu itererar vi över alla testloppen. Plockar de 24 högsta scorade hästarna__

__På VNUM__

In [ ]:
cm_list = []
b_list = []
d_list = df1.Datum.drop_duplicates().tolist()
for date in d_list:
    df_test1 = df1[df1.Datum == date].drop('Datum', axis = 1)
    df_test1['Pred'] = np.where(df_test1.VNUM.isin([1,2,3]),1,0)
    df_test1['Facit'] =  np.where((df_test1.Pred == 1) & (df_test1.Y == 1) ,1,0)
    cm_list.append(df_test1.Facit.sum())
    b_list.append(df_test1.Pred.sum())

In [ ]:

tp_tot = 0
out_tot = 0
for tp, tot in zip(cm_list,b_list):
    tp_tot = tp_tot + tp
    out_tot = out_tot + tot

avg = tp_tot / len(cm_list)
avg

In [ ]:
avg_out_tot = out_tot/len(cm_list) 
avg_out_tot


In [ ]:
# Precision
avg/avg_out_tot

In [ ]:
# Recall
avg/7

__Genom att plocka de tre högsta i VNUM för varje lopp så hittar vi 69% av de faktiska vinnarna. Bygg en modell på Leffes travstatstik för att hitta de som VNUM inte hittar. Modellen byggs på de hästar som inte ligger i VNUM 1,2 eller 3__

In [ ]:
df2 = df1[~df1.VNUM.isin([1,2,3])]    

__Nu plockar vi ut 52 (20%) v75 omgångar för att använda dem som test och utvärdera modellen på__

In [ ]:
v75 = df2.Datum.drop_duplicates().to_frame()

v75['is_test']=np.random.uniform(0,1,len(v75))<=0.2

test, basedf = v75[v75['is_test']==True], v75[v75['is_test']==False]

In [ ]:
# Plockar in imgångarna vi ska träna modellen på
train = df2[df2.Datum.isin(basedf.Datum.tolist())]

# De 52 vi utvärderar på 
validate = df2[df2.Datum.isin(test.Datum.tolist())]

In [ ]:
num_attribs = ['VLP',
 'SVLP',
 'VSVLP',
 'VPN_SUM',
 'VPN_SUM_ORD',
 'VPK_SUM',
 'VPK_SUM_ORD',
 'VLPB',
 'SVLPB',
 'VSVLPB',
 'E_P',
 'E_P_Num',
 'E_N',
 'E_R',
 'E_U',
 'G_R',
 'A_R',
 'T_R',
 'ToR',
 'Ts_R']

In [ ]:
# Träningsdata
# Alla förklaringsvaribler i en multidimensionell array där kategrisvaribler har gjorts om till
# dummyvariabler
features_train = full_pipeline.fit_transform(train)
## En array som håller det vi vill predikter
label_train = train["Y"].copy()

In [ ]:
# Valideringsdata
# Alla förklaringsvaribler i en multidimensionell array där kategrisvaribler har gjorts om till
# dummyvariabler
features_valid = full_pipeline.fit_transform(validate)
## En array som håller det vi vill predikter
label_valid = validate["Y"].copy()

In [ ]:
param_grid = { 
    'n_estimators': [500],
    'max_leaf_nodes':[16],
    'max_features': ['sqrt', 'auto','log2'],
    'criterion': ['gini'],
    'bootstrap': [True, False],
    'n_jobs':[-1]
    
}

In [ ]:
# Skapar instansen av modellen
rf_all = RandomForestClassifier()

# Instansen av gridsearc
grid_search = GridSearchCV(rf_all, param_grid, cv=5, scoring = 'roc_auc' , return_train_score=True)
# Hyperparameter oprimering
grid_search.fit(features_train, label_train)

In [ ]:
pararg = grid_search.best_params_
pararg

In [ ]:
# Utvärderar styrkan i modellen - sätter hyperparametrarna och cross fold fem
rf_mod_6 = RandomForestClassifier(**pararg)
scores_opt = cross_val_score(rf_mod_6, features_train, label_train , scoring = "roc_auc", cv = 5 ) 

In [ ]:
scores_opt

In [ ]:
scores_opt.mean()

In [ ]:
# Bygger den slutliga modellen för random forrest
rf_mod_6.fit(features_train,label_train)

__Nu testar vi att bygga en enkel ensamble model__

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import neighbors
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

n_neighbors = 25

log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(**pararg)
svm_clf = SVC(gamma="auto", random_state=42, probability = True)
kne_clf = neighbors.KNeighborsClassifier(n_neighbors, weights='uniform')


In [ ]:
LogisticRegression?

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf), ('kn', kne_clf)],
    voting='soft')

In [ ]:
voting_clf.fit(features_train,label_train)

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, kne_clf, voting_clf):
    clf.fit(features_train,label_train)
    predict = clf.predict_proba(features_valid)
    fpr, tpr, threshold = roc_curve(label_valid,predict[:,1])
    roc_auc = auc(fpr,tpr)
    print(clf.__class__.__name__, roc_auc)

In [ ]:
predict = rf_mod_6.predict_proba(features_valid)

In [ ]:
fpr, tpr, threshold = roc_curve(label_valid,predict[:,1])

roc_auc = auc(fpr,tpr)
print(roc_auc)

__Nu plockar vi de två högst scorade hästarna i varje lopp och utvärderar på valideringsdatat__

In [ ]:
def top(df):
    return df.sort_values(['Lopp','Prob1'], ascending = [True, False])[['Prob1','Lopp','Y']].iloc[:14]

In [ ]:
d_list = validate.Datum.drop_duplicates().tolist()
res_list = []
for date in d_list:
    df_test1 = validate[validate.Datum == date].drop('Datum', axis = 1)
    df_test1['GRUPP'] = df_test1.GRUPP.astype('object')
    features = full_pipeline.fit_transform(df_test1)
    predict = rf_mod_6.predict_proba(features)
    predict_frame = pd.DataFrame({'Prob0':predict[:,0],'Prob1':predict[:,1]})
    df_lopp = predict_frame.merge(df_test1.reset_index(), right_index = True, left_index = True)
    grouped = df_lopp.groupby('Lopp')
    result = grouped.apply(top).Y.sum()
    res_list.append(result)

In [ ]:
tot = 0
for res in res_list:
    tot +=res
avg = tot/len(res_list)
avg

In [ ]:
4.8 + 2.1

In [ ]:
# Precision

6.9 / (21 + 14)

In [ ]:
6.9 / 7

__Tar fram frekvenser av utfallet__

In [ ]:
# Tar fram diagram på fördelningen. True-Positive
# Gör en dataframe för att sedan gruppera

tp = {'TP':res_list}
tp_frame = pd.DataFrame.from_dict(tp)

In [ ]:
tp_frame.TP.value_counts().sort_index()

In [ ]:
dist = round(tp_frame.TP.value_counts().sort_index()/len(test)*100,1)

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(12,8))

ax1.bar(koll.index.tolist(), dist, color=(0.2, 0.4, 0.6, 1))

for x, y in zip(koll.index.tolist(), koll):
    ax1.text(x,y+0.5,str(int(round(y)))+'%', ha = 'center', fontsize=14)
    
ax1.set_title('Utvärderat på 35 omgångar - 14 hästar uttagna i varje omgång, VNUM 1-3 exkluderat')

ax1.set_ybound(0,60)

save_fig('DistRes25')

__Nu bygger vi en modell på hela datamängden med de optimerade hyperparametrarna och utvärderer med cross fold__

In [ ]:
from sklearn.externals import joblib

# Pipelineobjekt
joblib.dump(full_pipeline, 'Pipeline_v6.pkl')

# Modellobjekt
joblib.dump(rf_mod_6, 'Travmodel_v6.pkl')